#### **Formulating mTSP through MTZ**

MTZ is being selected as the family of Subtour Elimination Constraints (SECs) posed by alternative popular formulations such as DFJ as constraints of DFJ increase exponentially with increasing number of nodes (Bektas, 2006).

**Define a binary variable indicating if an arc is used or unused in a tour:**

$$x_{ij} = 1$$

if arc $(i,j)$ is used on the specific tour, and: 

$$x_{ij} = 0$$

if arc $(i,j)$ is not used on the specific tour.

**Minimize the sum of the cost of the tour:**

$$ minimize \: \Sigma^n_{i=1}\Sigma^n_{j=1} c_{ij}x_{ij}$$

where $C = c_{ij}$ is some distance matrix associated with a set of arcs between the potential "cities" of the mTSP problem

$$s.t. \\ \Sigma^n_{j=1}x_{1j}=m$$

where $m$ is the number of salesmen in the problem. We want $m$ salesmen to leave the initial depot. Essentially we are guaranteeing that this is the single depot version of the problem; maybe this is worth revisiting later on.

**Ensure that m salesmen come back:**

$$\Sigma^n_{j=1}x_{j1}=m$$

**Only one connection is allowed from some city i to some other (non-starting) city j:**

$$\Sigma^n_{i=1}x_{ij}=1, \: j=2,...,n$$

If we don't do this we would allow stuff like multiple routes going from one city to another. This is not good.

**Only one connection is allowed from some city j to some other (non-starting) city i:**

$$\Sigma^n_{j=1}x_{ij}=1, \: i=2,...,n$$

If we don't do this we would allow stuff like multiple routes going from one city to another. This is not good. Combination of last two also prevents things like creating cyclical routes where we go around different cities repeatedly in a circle.

**Make sure everything belongs to the proper set of arcs, make sure no unexpected variable assignments occur:**

$$x_{ij} \in \{0,1\}, \: \forall(i,j) \in A$$

**Subtour elimination constraints:**

We're gonna use MTZ here because it makes for a less complex encoding, though do note that DFJ is a "stronger" formulation, boiling down to having a smaller solution space. Why? I don't really know. Don't ask my why I could not tell you. Anyways:

**SEC FOR mTSP:**

$$u_i - u_j + px_{ij} \leq p - 1$$

p denotes maximum number of nodes which can be visited by a salesman. Node potentials (u's) denote the particular order of visitation. Think of $u_i$ as the node we're on, and $u_j$ as the node we are going to next. Obviously, $u_j$ must be above $u_i$, otherwise the path would "connect in on itself", creating a subtour/cycle. Note that the depot does not have an assigned $u_i$ at the beginning, as that would result in it being impossible for the salesman to return back to the depot.

The above can also be denoted as:

$$u_1 = 1$$
$$\forall i \neq 1: \: 2 \leq u_i \leq n$$
$$\forall \neq 1, \forall j \neq 1 : \: u_i - u_j + 1 \leq (n-1)(1-x_{ij})$$

if $x_{ij} = 1$, i.e. the tour does actually get used, then:

$$u_j \geq u_i + 1$$

This is pretty much exactly what we want. Consider the other case, where $x_{ij} = 0$

$$u_i - u_j + 1 \leq n - 1$$

This is also correct, since we know that the largest difference between two indexes of any (non-depot) nodes does equal to $n - 1$

i.e. if there are ten nodes, the maximum difference between the nodes is 9.

In [1]:
# imports
import random
import gurobipy as grb
import numpy as np
import math

#important global params.
n_cities = 10
n_salesmen = 3

In [2]:
# Generate set of random points deterministically => straightforward.
def generate_random_coordinates(num_points, seed = 158):
    random.seed(seed)
    coordinates = [(random.uniform(-1, 1), random.uniform(-1, 1)) for _ in range(num_points)]
    return coordinates

In [3]:
# Generate distance matrix
points = generate_random_coordinates(num_points=n_cities)

C = np.zeros((n_cities,n_cities))

for i in range(0, n_cities):
    for j in range(0, len(points)):
        C[i,j] = math.dist(points[i], points[j])

In [5]:
n = range(0, n_cities)
m = range(0, n_cities)

set_I = n
set_J = m

# Set up a Gurobi optimizer, name it accordingly.
opt_model = grb.Model(name="mTSP Optimizer")

# X, denoting if a route is used or not, is a binary variable. Sets up our connection of matrices.
x_vars = {(i,j):opt_model.addVar(vtype=grb.GRB.BINARY,
            name="x_{0}_{1}".format(i,j)) 
            for i in set_I for j in set_J}

# u variables, tracking order of different visited nodes.
u_vars  ={(i):opt_model.addVar(vtype=grb.GRB.INTEGER,
            name="x_{0}".format(i)) 
            for i in n}

## The constraints - naming conventions follow Bektas, 2006:
opt_model.addConstr(grb.quicksum((x_vars[0,j] for j in range(1, n_cities))) == n_salesmen, name='const_1_1')
opt_model.addConstr(grb.quicksum((x_vars[j,0] for j in range(1, n_cities))) == n_salesmen, name='const_2')
opt_model.addConstr(grb.quicksum((x_vars[i,j]for i in set_I
                            for j in range(1, n_cities))) == 1, name='const_3')
opt_model.addConstr(grb.quicksum((x_vars[i,j]for j in set_J
                            for i in range(1, n_cities))) == 1, name='const_4')

## Subtour elimination constraints:
opt_model.addConstrs((u_vars[i] >= 2 for i in range (1, n_cities)), name='uconst_1')
opt_model.addConstrs((u_vars[i] <= n_cities for i in range (1, n_cities)), name='uconst_2')
opt_model.addConstr((u_vars[1] == 0), name='uconst_3')

## The main SEC constraint:
for i in range(0, n_cities):
    for j in range(0, n_cities):
        if i != j and 1 <= i and i <= n_cities - 1:
            opt_model.addConstr(u_vars[i] - u_vars[j] + n_cities * x_vars[i, j] <= n_cities - 1)

#This is the worst way of expressing this known to mankind - but whatever.
opt_model.setObjective(grb.quicksum(C[a,b]*x_vars[a,b] for a in range (0, n_cities)
                                    for b in range(0, n_cities)), grb.GRB.MINIMIZE)

#Now optimize the model
opt_model.optimize()


range(1, 11)


KeyError: (0, 1)